In [264]:
import sympy as sp
from sympy.physics.quantum import TensorProduct
import numpy as np
digits=100
def norm(vec):
    i=0
    for ele in vec:
        i = abs(ele)**2 + i
    return i
def annihilation(n):
    def anni_(i, j):
        if j-i == 1:
            return sp.sqrt(j)
        else:
            return 0
    return sp.Matrix(n, n, anni_)
def creation(n):
    def crea_(i, j):
        if i-j == 1:
            return sp.sqrt(i)
        else:
            return 0
    return sp.Matrix(n, n, crea_)
def get_H():
    dim = 20
    Q_dim=6
    g=0.1*2*sp.pi
    anharmonicity =-0.225
    a_dag = creation(dim)
    a = annihilation(dim)
    b_dag=creation(Q_dim)
    b=annihilation(Q_dim)
    A=TensorProduct(a,sp.eye(Q_dim))
    A_dag=TensorProduct(a_dag,sp.eye(Q_dim))
    B=TensorProduct(sp.eye(dim),b)
    B_dag=TensorProduct(sp.eye(dim),b_dag)
    H_trans = 1 / 2 * anharmonicity * ((b_dag*b)*(b_dag*b-sp.eye(Q_dim)))
    H0=g*(TensorProduct(a_dag,b)+TensorProduct(a,b_dag))+TensorProduct(sp.eye(dim),H_trans)
    H=-1j*(H0+0.5*2*sp.pi*(B_dag*B+B+B_dag+1j*(B-B_dag)))
    vec=1/sp.sqrt(dim*Q_dim)*sp.ones(dim*Q_dim,1)
    vec=vec.evalf(digits)
    return H.evalf(digits),vec
def get_auxiliary(t):
    dim = 20
    Q_dim=6
    g=0.1*2*sp.pi
    anharmonicity =-0.225
    a_dag = creation(dim)
    a = annihilation(dim)
    b_dag=creation(Q_dim)
    b=annihilation(Q_dim)
    A=TensorProduct(a,sp.eye(Q_dim))
    A_dag=TensorProduct(a_dag,sp.eye(Q_dim))
    B=TensorProduct(sp.eye(dim),b)
    B_dag=TensorProduct(sp.eye(dim),b_dag)
    H_trans = 1 / 2 * anharmonicity * ((b_dag*b)*(b_dag*b-sp.eye(Q_dim)))
    H0=g*(TensorProduct(a_dag,b)+TensorProduct(a,b_dag))+TensorProduct(sp.eye(dim),H_trans)
    H=-1j*(H0+0.5*2*sp.pi*(B_dag*B+B+B_dag+1j*(B-B_dag)))
    A=sp.Matrix([[t*H,-1j*(B+B_dag)],[0*H,t*H]])
    vec=1/sp.sqrt(dim*Q_dim)*sp.ones(dim*Q_dim,1)
    vec=sp.Matrix([0*vec,vec])
    return A.evalf(digits),vec.evalf(digits)
def expm(H,vec):
    b=vec
    c=vec
    j=1
    while(1):
        b=(H*b/j).evalf(digits)
        c=(c+b).evalf(digits)
        if max(abs(np.array(b,dtype=np.complex128))) <= 10**-100:
            break
        j=j+1
    return c

In [265]:
ts = np.array([1,2,3,4,5,6,7,8,9,10])
ref = {"state_propagator":[],"state_propagator_der":[],"gate_propagator":[],"gate_propagator_der":[]}

In [266]:
for t in ts:
    H,vec = get_H()
    ref["state_propagator"].append(expm(t*H,vec))
for t in ts:
    H,vec = get_auxiliary(t)
    ref["state_propagator_der"].append(expm(H,vec))

KeyboardInterrupt: 

In [259]:
digits=300
def get_H(N):
    control=get_control(N)
    control_pra=0.5*2*sp.pi*sp.ones(len(control),1)
    H_control=control_H(control_pra,control)
    H=-1j*(H_control+get_int(N)*0.1*2*sp.pi)
    vec=1/sp.sqrt(2**N)*sp.ones(2**N,1)
    return H.evalf(digits),vec.evalf(digits)
def get_control(N,):
    sigmap=creation(2,)
    sigmam =annihilation(2,)
    sigmap=sigmap
    sigmam=sigmam
    sigmax=sigmap+sigmam
    sigmay=-1j*sigmap+1j*sigmam
    control=[]
    if N==1:
        control.append(TensorProduct(sigmax, sp.eye(2 ** (N - 1))))
        control.append(TensorProduct(sigmay, sp.eye(2 ** (N - 1))))
        return control
    else:
        a=sp.eye(2**(N-1))
        control.append(TensorProduct(sigmax,a))
        control.append(TensorProduct(sigmay, sp.eye(2 ** (N - 1))))
        for i in range(1,N-1):
            control.append(TensorProduct(TensorProduct(sp.eye(2**i),sigmax), sp.eye(2 ** (N - 1-i))))
            control.append(TensorProduct(TensorProduct(sp.eye(2 ** i), sigmay), sp.eye(2 ** (N - 1 - i))))
        control.append(TensorProduct(sp.eye(2**(N-1)),sigmax))
        control.append(TensorProduct(sp.eye(2**(N-1)),sigmay))
    return control
def control_H(control,H_control):
    H=0*H_control[0]
    for i in range(len(control)):
        H=H+control[i]*H_control[i]
    return H
def get_int(N):
    sigmap=creation(2)
    sigmam = annihilation(2)
    sigmaz=sigmap*(sigmam)
    H0=sp.zeros(2**N)
    SIGMAZ=TensorProduct(sigmaz,sigmaz)
    H0=H0+TensorProduct(SIGMAZ,sp.eye(2**(N-2)))+TensorProduct(sp.eye(2**(N-2)),SIGMAZ)
    for i in range(1,N-2):
        H0=H0+TensorProduct(TensorProduct(sp.eye(2**i),SIGMAZ),sp.eye(2 ** (N - 2 - i)))
    return H0
def get_auxiliary(N,t):
    H,vec = get_H(N)
    control=get_control(N)
    vec = sp.Matrix([0*vec,vec])
    A = sp.Matrix([[t*H,-1j*control[3]],[0*H,t*H]])
    return A.evalf(digits), vec.evalf(digits)

In [260]:
N=7
for t in ts:
    H,vec = get_H(N)
    ref["gate_propagator"].append(expm(t*H,vec))
for t in ts:
    H,vec = get_auxiliary(N,t)
    ref["gate_propagator_der"].append(expm(H,vec))

In [263]:
ref["gate_propagator_der"][0]

Matrix([
[ 0.499999861836709272852485738210192229676979079270062849765710932057840772186522847898542036859645129373783257504763424835395548468361513834002803361819713022053208732224849767412027585036973351477553417590826800892245954781079210865506725596612686536910158952178579098069847437010641833252685917904535 - 0.000157020392113525445279186250282846012819500680225656800350967561504076586648702014704800678883716594406040600755187659865187713101164348277859734314648835822671401290458952969452248187259268842216761596318366256481134674826286232184297081942140736834140731200490176884949408737935585239646838638407812*I],
[-0.500314123033298891069384494065694724175440519167941917540777689123037804093190216803165946905615191663170254369305149444489355950788433449979637234930048319853542595730515008978917706232235382999785439854094309078485613179959181057160566946318948207668850184357670302771233675481642404233182180096049 + 0.00015714871049888843244349983340171579290524301120815590395717808